# Build commit table

The overall goal is to pull commit information about a single project from github, then format that info as a table where each row is a separate commit. Here is the target, where each row represents a single commit:

<img src="https://www.dropbox.com/s/om587xsel6duayq/Screenshot%202019-06-25%2012.20.00.png?raw=1">

I first tried to run this from colab. I was able to download the hpcl folder to my desktop from here: https://github.com/HPCL/ideas-uo/tree/master/code/hpcl. Then upload it with code below. I first zipped it on the desktop so you can see unzipping below.

However, I failed to bring in mysql.connector from colab so switched to jupyter server on my desktop

In [1]:
#!rm -r hpcl*

In [2]:
#from google.colab import files
#files.upload();

Even when on jupyter, could not get mysql installed so commented out for now.

In [3]:
import os, sys, subprocess
import hpcl.Command
from hpcl import GitCommand
#import mysql.connector    #works from command line but not here
from hpcl.analysis import *



## Biggish issue

Jason's code will not run in 3.6 so I switched kernels to 2.7. Reluctantly! Like to get everyhing in 3.6 soon.

Note I did have to create a tmp dir prior to running this.

In [4]:
#Thanks to Jason for this code

def checkoutSubrepos(repos,tdir):
    #currdir = os.getcwd()
    for repopath in repos.keys():
        if repopath == tdir: continue
        retcode, out, err = Command.Command('git checkout %s%s' % repos[repopath]).run(dryrun=dry_run)
    return


if __name__ == "__main__":
    
    print('Going to load repos...')

    #Load the list of repos to clone from the following file
    urls = open(os.path.join('.','URL.txt'),'r').readlines()

    #Move to the tmp directory that will hold the repos
    currdir = os.getcwd()
    tmpdir = os.path.join(currdir,'tmp')  #sff: fails if does not already exist
    #os.chdir(tmpdir)

    commander = GitCommand.GitCommand(tmpdir)

    #Download each repo
    for url in urls:
        print('Cloning: ' + url)
        commander.cloneRepo(url)

    print('Finished cloning repos.')

    outfile = open('../stats.csv','w')
    outfile.write('Date' + ',' + ','.join(categories.keys()) + ',' + 'Other' + '\n')

    #Setup DB connection, this will probably change soon.
    #mydb = mysql.connector.connect(host="localhost", port="3307", user="pythondb", passwd="********", database="gitstats")
    #print(mydb)
    #mycursor = mydb.cursor()
    #sql = "INSERT INTO stats (reponame, stats) VALUES(%s, %s)"

    #Now run commands on the repos
    for repo in os.listdir('.'):

        print('Checking: ' + repo)
        prefix,versions = commander.getRepoVersions(repo)
        print(prefix)
        print(versions)

        #Gets all the commit data in a dictionary format {authorname:{'total_commits':0, 'commits':{'date':'', 'message':'', 'diffs':{'filename':'', 'diff':''}}}}
        commits = commander.getRepoCommitData(repo)

        print(len(commits))
        for key,values in commits.items():
            print(key)
            #print(values)

        """ stats = getStats('.',repo)
        ts = repo + ', ' + out.strip().split('-')[0] 
        buf = ts
        for i in range(0,len(category_names)): 
            buf += ', %d' % stats[category_names[i]]
            linecounts[i].append(stats[category_names[i]]*0.001)
        outfile.write(buf+'\n')
        #years.append(int(ts))
        print(stats)
        
        val = (repo, buf)"""
        #mycursor.execute(sql,val)
        #mydb.commit()

    outfile.close()




Going to load repos...
Cloning: https://github.com/spack/spack.git

Cloning: https://github.com/lanl/LATTE.git

Finished cloning repos.
Checking: LATTE

['cf192f6fd4209d7dc0759321451c30ccf5d51b02', '79d950d1f714c1358a018210f8f2603edef84520', 'b191c3c3fbfb699bded53435611d24f27fffcd2f', '8eb5f8188764116ec3a1690233cee51a5d636583']
git checkout cf192f6fd4209d7dc0759321451c30ccf5d51b02

git checkout 79d950d1f714c1358a018210f8f2603edef84520

git checkout b191c3c3fbfb699bded53435611d24f27fffcd2f

git checkout 8eb5f8188764116ec3a1690233cee51a5d636583

15
Christoph Junghans <junghans@votca.org>
Christian Negre <cnegre@lanl.gov>
Marc Cawkwell <cawkwell@lanl.gov>
cawkwell <cawkwell@gransasso.lanl.gov>
Christoph Junghans <junghans@lanl.gov>
rperriot <rperriot@lanl.gov>
Nestor F. Aguirre <nfaguirrec@gmail.com>
Christoph Junghans <christoph.junghans@gmail.com>
Axel Kohlmeyer <akohlmey@gmail.com>
a1k12 <a1k2112@gmail.com>
Chang Liu <chliu1990@users.noreply.github.com>
Christian F. A. Negre <christian

## Let's see what we have

In [5]:
len(commits)  #50 separate people doing commits on the project

50

In [6]:
type(commits)

dict

In [7]:
tups = commits.items()  #get it in form easier to see

In [8]:
tups[0]

('Bob Robey <brobey@lanl.gov>',
 {'commits': [{'date': 'Sat Oct 25 21:59:01 2014 -0600',
    'diffs': [{'diff': '     """ImageMagick is a image processing library"""',
      'filename': '16 +3,16 @@ from spack import *'}],
    'message': '    Fixing errors in depends_on and updating version for ImageMagick\n'},
   {'date': 'Sat Oct 25 21:25:25 2014 -0600',
    'diffs': [{'diff': '+',
      'filename': 'var/spack/packages/ImageMagick/package.py'}],
    'message': '    Adding autotools and ImageMagick and patch for mpe2\n'},
   {'date': 'Sat Oct 25 21:59:01 2014 -0600',
    'diffs': [{'diff': '     """ImageMagick is a image processing library"""',
      'filename': '16 +3,16 @@ from spack import *'}],
    'message': '    Fixing errors in depends_on and updating version for ImageMagick\n'},
   {'date': 'Sat Oct 25 21:25:25 2014 -0600',
    'diffs': [{'diff': '+',
      'filename': 'var/spack/packages/ImageMagick/package.py'}],
    'message': '    Adding autotools and ImageMagick and patch

In [9]:
x = tups[1]
x

('Gregory L. Lee <lee218@cab687.llnl.gov>',
 {'commits': [{'date': 'Sat Dec 19 07:43:34 2015 -0800',
    'diffs': [{'diff': "     version('4.0.0', 'd00301c078cba57ef68613be32ceea2f')",
      'filename': '6 +7,7 @@ class Mrnet(Package):'}],
    'message': '    added mrnet 5.0.1\n'},
   {'date': 'Wed Dec 9 14:10:05 2015 -0800',
    'diffs': [{'diff': ' ', 'filename': '4 +1,5 @@'}],
    'message': '    install python files to libxml2 prefix instead of python prefix and ignore non-python files when activating\n'},
   {'date': 'Sat Dec 19 07:43:34 2015 -0800',
    'diffs': [{'diff': "     version('4.0.0', 'd00301c078cba57ef68613be32ceea2f')",
      'filename': '6 +7,7 @@ class Mrnet(Package):'}],
    'message': '    added mrnet 5.0.1\n'},
   {'date': 'Wed Dec 9 14:10:05 2015 -0800',
    'diffs': [{'diff': ' ', 'filename': '4 +1,5 @@'}],
    'message': '    install python files to libxml2 prefix instead of python prefix and ignore non-python files when activating\n'}],
  'total_commits': 4})

In [10]:
x[1]['commits']

[{'date': 'Sat Dec 19 07:43:34 2015 -0800',
  'diffs': [{'diff': "     version('4.0.0', 'd00301c078cba57ef68613be32ceea2f')",
    'filename': '6 +7,7 @@ class Mrnet(Package):'}],
  'message': '    added mrnet 5.0.1\n'},
 {'date': 'Wed Dec 9 14:10:05 2015 -0800',
  'diffs': [{'diff': ' ', 'filename': '4 +1,5 @@'}],
  'message': '    install python files to libxml2 prefix instead of python prefix and ignore non-python files when activating\n'},
 {'date': 'Sat Dec 19 07:43:34 2015 -0800',
  'diffs': [{'diff': "     version('4.0.0', 'd00301c078cba57ef68613be32ceea2f')",
    'filename': '6 +7,7 @@ class Mrnet(Package):'}],
  'message': '    added mrnet 5.0.1\n'},
 {'date': 'Wed Dec 9 14:10:05 2015 -0800',
  'diffs': [{'diff': ' ', 'filename': '4 +1,5 @@'}],
  'message': '    install python files to libxml2 prefix instead of python prefix and ignore non-python files when activating\n'}]

I am going to reformat the data. Currently the key is user and the value is a nested structure that includes all the commits for that person. I want to invert this. I'd like a list of commits, where each commit has info including person.

Note Python is a mess when it comes to time zones. What is supposed to work, %z, does not. See discussion here: https://stackoverflow.com/questions/3305413/python-strptime-and-timezones.

In particular, these fail when include UTC: '%a %b %m %X %y %z' and '%c %z'.

I am resorting to a utility package.

In [11]:
import datetime
from dateutil import parser
parser.parse("Tue Aug 2 14:58:31 2016 +0200")  #+0200 hours => 7200 seconds

datetime.datetime(2016, 8, 2, 14, 58, 31, tzinfo=tzoffset(None, 7200))

In [12]:
all_commits = []
for entry in tups:
    name = entry[0]                               #'Gilles Fourestey <gilles.fourestey@epfl.ch>'
    commit_list = entry[1]['commits']
    for a_commit in commit_list:
        date = parser.parse(a_commit['date'])         #"Tue Aug 2 14:58:31 2016 +0200"
        files = []
        diffs = a_commit['diffs']
        for diff_dict in diffs:
            filename = diff_dict['filename']          #'var/spack/repos/builtin/packages/plumed/package.py'
            files.append(filename)
        message = a_commit['message']             #'Fixed broken link in README (#1733)\n'
        new_dict = {'date': date, 'files': files, 'message': message, 'person': name}
        all_commits.append(new_dict)
        
    

In [22]:
len(all_commits) #2756 commits spread over roughly 3 years

2756

In [13]:
all_commits[0]

{'date': datetime.datetime(2014, 10, 25, 21, 59, 1, tzinfo=tzoffset(None, -21600)),
 'files': ['16 +3,16 @@ from spack import *'],
 'message': '    Fixing errors in depends_on and updating version for ImageMagick\n',
 'person': 'Bob Robey <brobey@lanl.gov>'}

Looks ok to me. Now converting to pandas dataframe is easy peasy.

In [14]:
import pandas as pd

In [15]:
df_commits = pd.DataFrame(all_commits)  #gotta love pandas

In [16]:
df_commits.head()

,date,files,message,person
0,2014-10-25 21:59:01-06:00,"[16 +3,16 @@ from spack import *]",Fixing errors in depends_on and updating v...,Bob Robey <brobey@lanl.gov>
1,2014-10-25 21:25:25-06:00,[var/spack/packages/ImageMagick/package.py],Adding autotools and ImageMagick and patch...,Bob Robey <brobey@lanl.gov>
2,2014-10-25 21:59:01-06:00,"[16 +3,16 @@ from spack import *]",Fixing errors in depends_on and updating v...,Bob Robey <brobey@lanl.gov>
3,2014-10-25 21:25:25-06:00,[var/spack/packages/ImageMagick/package.py],Adding autotools and ImageMagick and patch...,Bob Robey <brobey@lanl.gov>
4,2014-10-25 21:59:01-06:00,"[16 +3,16 @@ from spack import *]",Fixing errors in depends_on and updating v...,Bob Robey <brobey@lanl.gov>


In [17]:
#df_sorted = df_commits.sort('date')  #this fails in 2.7

In [18]:
df_sorted = df_commits.sort_values(by='date')
df_sorted = df_sorted.reset_index(drop=True)

In [19]:
df_sorted.head()

,date,files,message,person
0,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
1,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
2,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
3,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
4,2013-02-18 23:46:04-08:00,"[,7 +19,7 @@ import spack]","Dependencies now work. Added libelf, libd...",Todd Gamblin <tgamblin@llnl.gov>


Write it out to file

In [23]:
df_sorted.to_csv('sorted_commits.csv')  #ideas-uo-master/code/tmp/spack

In [21]:
len(df_sorted)

2756